# Classification d'image

https://github.com/pytorch/examples/blob/master/mnist/main.py

http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf

https://www.superdatascience.com/blogs/the-ultimate-guide-to-convolutional-neural-networks-cnn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

image Net !

- Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias, padding_mode) : voir [Conv2d](https://pytorch.org/docs/stable/nn.html#conv2d)
     - Applique une convolution 2D sur un signal d'entrée composé de plusieurs plans d'entrée.
     - la convolution est une fonction dérivée de deux fonctions données par intégration qui exprime comment la forme de l'une est modifiée par l'autre

![Conv2d](https://github.com/vdumoulin/conv_arithmetic/blob/master/gif/no_padding_no_strides.gif?raw=true)

![Conv 1](https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/35_blog_image_10.png)

![Conv 2](https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/35_blog_image_11.png)

- ReLU() : voir [ReLu](https://pytorch.org/docs/stable/nn.html#relu)
    - Applique la fonction d'Unité de Rectification Linéaire : ReLU(x)=max(0,x)
    
![ReLu](https://pytorch.org/docs/stable/_images/ReLU.png)



- MaxPool2d(kernel_size, stride, padding, dilation, return_indices, ceil_mode) : voir [MaxPool2d](https://pytorch.org/docs/stable/nn.html#maxpool2d)
    - Applique un Max Pooling 2D sur un signal d'entrée composé de plusieurs plans d'entrée.
    - permet d'évité un surapprentissage
        

![max_pooling](https://upload.wikimedia.org/wikipedia/commons/e/e9/Max_pooling.png)



- Linear(in_features, out_features, bias) : voir [Linear](https://pytorch.org/docs/stable/nn.html#linear)
    - Applique une transformation linéaire aux données entrantes

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

- définition fonction loss + trouver la bonne ! [wiki](https://fr.wikipedia.org/wiki/R%C3%A9seau_neuronal_convolutif)

    La couche de perte spécifie comment l'entrainement du réseau pénalise l'écart entre le signal prévu et réel. Elle est normalement la dernière couche dans le réseau.

    Diverses fonctions de perte adaptées à différentes tâches peuvent y être utilisées :
    - La perte « Softmax »est utilisée pour prédire une seule classe parmi K classes mutuellement exclusives.
    - La perte par entropie croisée sigmoïde est utilisée pour prédire K valeurs de probabilité indépendante dans [0,1].
    - La perte euclidienne est utilisée pour régresser vers des valeurs réelles dans [-inf ,inf].

The negative log likelihood loss -> permet de maximiser l'erreur quand le bon label a une proba faible et de la diminuer quand bon label a proba élevé !

- backpropagation
    - l'erreur calculer se propage dans les première couche du réseau

![backpropagation](https://d3i71xaburhd42.cloudfront.net/db39fd79bb591b04d33207992f6ccde03cabd861/7-Figure1-1.png)

In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # The negative log likelihood loss.
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
import numpy as np

In [ ]:
torch.manual_seed(1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


train_batch_size = 64
test_batch_size = 1000
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


#dataset_train = datasets.MNIST('images', train=True, download=True, transform=transform)
#dataset_test = datasets.MNIST('images', train=False, transform=transform)
#
#train_loader1 = torch.utils.data.DataLoader(dataset_train,
#                                            batch_size=train_batch_size,
#                                            shuffle=True, **kwargs)
#
#test_loader1 = torch.utils.data.DataLoader(dataset_test,
#                                           batch_size=test_batch_size,
#                                           shuffle=True, **kwargs)

# random_split
dataset = datasets.MNIST('images', train=True, download=True, transform=transform)

nb_train = int(0.8 * len(dataset))
nb_test = len(dataset) - nb_train

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [nb_train, nb_test])

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=train_batch_size,
                                          shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=test_batch_size, 
                                          shuffle=True, **kwargs)

In [ ]:
print("The train set contains {} images, in {} batches".format(len(train_loader.dataset), len(train_loader)))
print("The test set contains {} images, in {} batches".format(len(test_loader.dataset), len(test_loader)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


epochs = 10
log_interval = 10
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

# save model
torch.save(model.state_dict(),"mnist_cnn.pt")